# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_load_db, ast_elt_load_db
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db
from asteroid_element import load_ast_elt, load_data_impl, convert_data
from astro_utils import anomaly_M2E, anomaly_E2f
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [4]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [5]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [6]:
fname_csv = hrzn_txt2csv(fname_txt)

In [7]:
fname_csv

'/ssd1/tmp/mysql/jpl/010_sun.csv'

In [8]:
df = hrzn_csv2df(fname_csv)

In [9]:
pd.set_option('display.max_rows', 10)

In [10]:
df

,BodyTypeCD,BodyNumber,BodyName,IntegrationSource,JD,CalendarDateTime,delta_T,qx,qy,qz,vx,vy,vz
0,S,10,Sun,DE431mx,2440400.5,1969-06-28,39.699541,0.004503,0.000810,-0.000061,-3.517495e-07,0.000006,-1.438024e-08
1,S,10,Sun,DE431mx,2440401.5,1969-06-29,39.702104,0.004502,0.000815,-0.000061,-3.587527e-07,0.000006,-1.432102e-08
2,S,10,Sun,DE431mx,2440402.5,1969-06-30,39.704668,0.004502,0.000821,-0.000061,-3.657021e-07,0.000006,-1.426221e-08
3,S,10,Sun,DE431mx,2440403.5,1969-07-01,39.707232,0.004501,0.000827,-0.000061,-3.725998e-07,0.000006,-1.420343e-08
4,S,10,Sun,DE431mx,2440404.5,1969-07-02,39.709795,0.004501,0.000832,-0.000061,-3.794481e-07,0.000006,-1.414425e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,S,10,Sun,DE431mx,2477596.5,2071-04-30,69.185505,-0.000708,-0.002510,-0.000002,4.158106e-06,-0.000004,-8.460845e-08
37197,S,10,Sun,DE431mx,2477597.5,2071-05-01,69.185494,-0.000704,-0.002514,-0.000002,4.164001e-06,-0.000004,-8.473888e-08
37198,S,10,Sun,DE431mx,2477598.5,2071-05-02,69.185482,-0.000700,-0.002518,-0.000002,4.169895e-06,-0.000004,-8.487000e-08
37199,S,10,Sun,DE431mx,2477599.5,2071-05-03,69.185470,-0.000696,-0.002522,-0.000002,4.175791e-06,-0.000004,-8.500189e-08


## Testing - Populate Database from CSV

In [11]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [12]:
engine = create_engine(connection_str)

In [13]:
engine.table_names()

['AsteroidElement',
 'AsteroidElement_Numbered',
 'AsteroidElement_Unnumbered',
 'HorizonsBody',
 'HorizonsImport',
 'HorizonsTime',
 'HorizonsVectors',
 'LargeBody',
 'SmallBody']

In [14]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [15]:
# ast_elt_load_db()

In [ ]:
ast_elt = load_ast_elt()

In [ ]:
ast_elt